In [1]:
import numpy as np
import gmsh

gmsh.initialize() #inicia Gmsh
gmsh.model.add('PROBLEMA2') #genera el modelo


# Generado de la figura
L1 = 10
L2 = 5
r = 1
#Lc = 20 #caso del ejercicio
Lc = 4
Lcr = 0.1  #Refina aun mas el mallado cerca del circulo

#chapa
p1 = gmsh.model.geo.addPoint(0, 0, 0, Lc)
p2 = gmsh.model.geo.addPoint(0, L2, 0, Lc) 
p3 = gmsh.model.geo.addPoint(L1, L2, 0, Lc) 
p4 = gmsh.model.geo.addPoint(L1, 0, 0, Lc)

#circulo
p5 = gmsh.model.geo.addPoint(0,r,0,Lcr)
p6 = gmsh.model.geo.addPoint(r,0,0,Lcr)


l1 = gmsh.model.geo.addLine(p5, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p6)

semi1 = gmsh.model.geo.addCircleArc(p5,p1 ,p6) 


C1 = gmsh.model.geo.addCurveLoop([l1, l2, l3, l4])
C2 = gmsh.model.geo.addCurveLoop([semi1])

S1 = gmsh.model.geo.addPlaneSurface([C1,C2]) #asi se genera la superifice descontando el circulo


#Generado de las condiciones de contorno

TrashNodo = gmsh.model.addPhysicalGroup(0, [p1])
gmsh.model.setPhysicalName(0, TrashNodo, 'Trash')

Empotradox = gmsh.model.addPhysicalGroup(1, [l1]) #1 es para curva o linea!
gmsh.model.setPhysicalName(1,Empotradox,'Empotradox')


Empotradoy = gmsh.model.addPhysicalGroup(1, [l4]) #1 es para curva o linea!
gmsh.model.setPhysicalName(1,Empotradoy,'Empotradoy')


Traccionado = gmsh.model.addPhysicalGroup(1, [l3])
gmsh.model.setPhysicalName(1,Traccionado,'Traccionado') #esta funcion le pone nombre a todos los nodos aca


Superficie = gmsh.model.addPhysicalGroup(2,[S1]) #2 es para superficies!
gmsh.model.setPhysicalName(2,Superficie, 'Superficie')


gmsh.model.geo.synchronize() #sincroniza lo que creamos con el modelo que creamos arriba
gmsh.model.mesh.generate(2)

In [2]:
NodeInfo = gmsh.model.mesh.get_nodes()
NodeInfo[1].shape
NumeroNodos = NodeInfo[0].shape[0]
MN = NodeInfo[1].reshape(NumeroNodos , 3)


ElementInfo = gmsh.model.mesh.get_elements()
ETAGS, ELEMENTS = gmsh.model.mesh.get_elements_by_type(2)
MC = ELEMENTS.reshape([ETAGS.shape[0],3])



In [3]:
entityTraccionada = gmsh.model.getEntitiesForPhysicalGroup(1, Traccionado)
Tgroup, Ttraccionada, Ltraccionada = gmsh.model.mesh.getElements(1, entityTraccionada[0])
Ltraccionada = Ltraccionada[0].reshape(Ttraccionada[0].shape[0],2)
espesor = 1
tension = 1000 #Pa
MNDFuerzas = np.zeros((2*NumeroNodos,1))

for l, linea in enumerate(Ltraccionada):
    n1 = int(linea[0] - 1)
    n2 = int(linea[1] - 1)
    longitud = np.abs(MN[n1,1]-MN[n2,1])
    Flocal = tension*longitud*espesor*1/2
    MNDFuerzas[int(2*n1)] += Flocal
    MNDFuerzas[int(2*n2)] += Flocal
    

In [4]:
Sx = 2 * (gmsh.model.mesh.get_nodes_for_physical_group(1,Empotradox)[0] - 1)
Sy = 2 * (gmsh.model.mesh.get_nodes_for_physical_group(1,Empotradoy)[0] - 1) +1
Strash1 = 2 * (gmsh.model.mesh.get_nodes_for_physical_group(0,TrashNodo)[0] - 1)
Strash2 = 2 * (gmsh.model.mesh.get_nodes_for_physical_group(0,TrashNodo)[0] - 1) + 1
S = np.hstack([Sx,Sy,Strash1,Strash2])
R = np.delete(np.arange(2*NumeroNodos),S) #mas elegante

In [5]:
E = 30E6
v = 0.3

D = (E/(1-v**2))*np.array([[1,v,0],[v,1,0],[0,0,(0.5*(1-v))]])
Matriz_coordenadas = np.ones((3,3))
Matriz_global = np.zeros([2*len(MN),2*len(MN)])
Matrices_Bloc = []
Areas = []

In [6]:
for l in range (len(MC)):
    
    I,M,J = MC[l]  
    
    i = int(I - 1) ; m = int(M - 1) ; j = int(J - 1)
    
    x = np.array([MN[i,0],MN[m,0],MN[j,0]])
    y = np.array([MN[i,1],MN[m,1],MN[j,1]])

    Bi = y[1] - y[2] ;   Bm = y[2] - y[0] ;    Bj = y[0] - y[1]
    Gi = x[2] - x[1] ;   Gm = x[0] - x[2] ;    Gj = x[1] - x[0]      
    Matriz_coordenadas[:,0] = x
    Matriz_coordenadas[:,1] = y
    
    Area = np.linalg.det(Matriz_coordenadas)/2
    
    Matriz1 = (1/(2*Area)) * np.array([[Bi,0,Bm,0,Bj,0],[0,Gi,0,Gm,0,Gj],[Gi,Bi,Gm,Bm,Gj,Bj]])
    Matrices_Bloc.append(Matriz1)
    Matriz_local = espesor * abs(Area) * (np.dot(np.transpose(Matriz1), np.dot(D,Matriz1)))

    A_G = np.array([i*2,i*2+1,m*2,m*2+1,j*2,j*2+1]) #aca pasa algo raro
    Matriz_global[np.ix_(A_G,A_G)] += Matriz_local
    
    

In [7]:
Desplazamientos_parcial = np.linalg.solve(Matriz_global[np.ix_(R,R)],MNDFuerzas[R])

Desplazamientos_f = np.zeros(len(MN)*2)

o = 0 

for i in range(NumeroNodos*2):
    if i in R:
        Desplazamientos_f[i] = Desplazamientos_parcial[o]
        o += 1
        
    if i in S:
        Desplazamientos_f[i] = 0
        
    
    
Fuerzas_total = np.dot(Matriz_global,Desplazamientos_f)

In [8]:
#Esfuerzos

Tensiones = []

for p in range(len(MC)):
    
    I,M,J = MC[p]    
    
    i = int(I - 1) ; m = int(M - 1) ; j = int(J - 1)

    
    nodos = [2*i,2*i+1,2*m,2*m+1,2*j,2*j+1]
    
    
    A = np.dot(Matrices_Bloc[p],Desplazamientos_f[nodos])
    
    B = np.dot(D,A)

   
    Tensiones.append(B)

Tensiones_ = np.array(Tensiones)

# for i in range (len(Tensiones)):
#    print(Tensiones[i])
#    print()

In [9]:
Problema2 = gmsh.view.add("Problema2")

problema2 = gmsh.view.addModelData(Problema2, 0, 'PROBLEMA2', 'ElementData', ETAGS, Tensiones_[:,0].reshape(-1,1) , numComponents=1)
#agrega todos los datos , el 0 viene de dependencias temporarles aca es 0 no varia con t



In [10]:
Desplazamientos = np.zeros((len(MN),3))

for i in range(len(MN)):
    Desplazamientos[i] = [Desplazamientos_f[i*2],Desplazamientos_f[i*2+1],0]

Problema12 = gmsh.view.add("Problema2")

problema12 = gmsh.view.addModelData(Problema12, 0, 'PROBLEMA2', 'NodeData', NodeInfo[0], Desplazamientos  , numComponents=3)
#agrega todos los datos , el 0 viene de dependencias temporarles aca es 0 no varia con t



In [11]:
gmsh.fltk.run() 

In [12]:

# for l in range (len(MC)):
    
#     I,M,J = MC[l]  
    
#     i = int(I - 1) ; m = int(M - 1) ; j = int(J - 1)
    
#     x = np.array([MN[i,0],MN[m,0],MN[j,0]])
#     y = np.array([MN[i,1],MN[m,1],MN[j,1]])
     
#     Matriz_coordenadas[:,0] = x
#     Matriz_coordenadas[:,1] = y
#     Matriz_coordenadas[:,2] = 0
    
#     print(Matriz_coordenadas[0,:])
#     print(Matriz_coordenadas[1,:])
#     print(Matriz_coordenadas[2,:])

[3.33333333 5.         0.        ]
[2.11455147 2.9598263  0.        ]
[1.17551611 3.77653487 0.        ]
[10.  5.  0.]
[10.   2.5  0. ]
[7.79989391 3.07657903 0.        ]
[2.45375518 1.65581189 0.        ]
[1.85205458 1.25493422 0.        ]
[1.47130993 1.84098199 0.        ]
[1.61467067 0.23219451 0.        ]
[1.44340577 0.36444572 0.        ]
[1.73515352 0.55261563 0.        ]
[0.72931374 2.1727652  0.        ]
[1.47130993 1.84098199 0.        ]
[0.92861494 1.73746305 0.        ]
[0.         1.50859114 0.        ]
[0.22693946 1.63832591 0.        ]
[0.16586897 1.38648828 0.        ]
[0.35089189 1.17284846 0.        ]
[0.41448705 1.40572738 0.        ]
[0.61836785 1.22092624 0.        ]
[3.54843433 2.47784548 0.        ]
[4.65792624 1.21953914 0.        ]
[3.2814982  1.26208221 0.        ]
[0.83584469 0.72964996 0.        ]
[1.00845397 0.7074606  0.        ]
[0.87827142 0.63890278 0.        ]
[0.48263806 3.20784052 0.        ]
[1.17551611 3.77653487 0.        ]
[1.07107102 2.75280168 0